# Soak Test Prototype

In [ ]:
test_case = "LVV-TXXXX"
test_exec = "LVV-EXXXX"

## Setup Environment

### Import Libraries

In [4]:
%load_ext autoreload
%autoreload 2

import os
import sys
import asyncio
import logging

import numpy as np

from matplotlib import pyplot as plt

from lsst.ts import salobj
from lsst.ts.observatory.control.maintel.mtcs import MTCS

from lsst.sitcom import vandv

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Get execution info 

In [ ]:
exec_info = vandv.ExecutionInfo()
print(exec_info)

### Confirm environment variables

In [ ]:
print(os.environ["OSPL_URI"])
print(os.environ["LSST_DDS_PARTITION_PREFIX"])
print(os.environ.get("LSST_DDS_DOMAIN_ID", "Expected, not set."))

### Setup logging

In [2]:
logging.basicConfig(format="%(name)s:%(message)s", level=logging.DEBUG)
log = logging.getLogger("soak_test")
log.level = logging.DEBUG
log.addFilter(vandv.logger.filter_dds_read_queue_is_filling)
log.addFilter(vandv.logger.filter_dds_read_queue_is_full)

In [3]:
domain = salobj.Domain()

mtcs = MTCS(domain=domain, log=log)
mtcs.set_rem_loglevel(40)

await mtcs.start_task

[None, None, None, None, None, None, None, None, None, None]

## Helper Functions

In [26]:
_az = 0
_el = 80

In [27]:
def random_walk_azel(n_steps, 
                     radius=3.5, 
                     min_az=-200., 
                     max_az=+200, 
                     min_el=30, 
                     max_el=75,
                     verbose=True,
                    ):

    if verbose:
        print(f"{'Steps':>10s}{'New Az':>10s}{'New El':>10s}")
        
    step = 0
    while step < n_steps:
        
        # current_az = mtcs.rem.mtmount.tel_azimuth.get()
        # current_az = current_az.actualPosition
        current_az = _az
        offset_az = np.sqrt(radius) * (2 * np.random.rand() - 1)
        new_az = current_az + offset_az
                
        # current_el = mtcs.rem.mtmount.tel_elevation.get()
        # current_el = current_el.actualPosition
        current_el = _el
        offset_el = np.sqrt(radius) * (2 * np.random.rand() - 1)
        new_el = current_el + offset_el
        
        if new_az <= min_az or new_az >= max_az:
            new_az = current_az - offset_az
            
        if new_el <= min_el or new_el >= max_el:
            new_el = current_el - offset_el

        if verbose:
            print(f"{step:10d}{new_az:10.2f}{new_el:10.2f}")

        yield new_az, new_el
        step += 1

In [28]:
for az, el in random_walk_azel(100):
    _az, _el = az, el

     Steps    New Az    New El
         0      0.34     81.77
         1      0.64     83.41
         2     -0.40     85.11
         3      0.53     84.48
         4      2.29     83.64
         5      1.22     83.55
         6      2.93     82.45
         7      3.09     82.41
         8      2.90     84.11
         9      3.77     84.37
        10      1.99     83.19
        11      0.72     83.15
        12      1.78     82.07
        13      1.14     80.64
        14     -0.31     80.42
        15      1.28     80.06
        16     -0.18     80.09
        17      1.22     78.75
        18      1.70     79.18
        19      2.06     80.01
        20      2.15     81.02
        21      3.84     79.55
        22      3.74     80.97
        23      3.78     80.88
        24      4.55     79.05
        25      6.13     79.47
        26      6.44     80.52
        27      4.92     79.29
        28      4.80     78.58
        29      4.62     79.84
        30      2.90     79.98
        